In [2]:
import ipynb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle
import scipy
import neurokit2 as nk
import random
import pyhrv

In [7]:
def get_bvpfeatures(data):
    features_bvp = None

    df, info = nk.ppg_process(data, sampling_rate=100)

    features = nk.ppg_intervalrelated(df)
    return features

In [1]:
def bvp_prep(data):
    data = data.fillna(method="ffill")
    data = data.fillna(method="bfill")
    data = data.drop(['HRV_LF','HRV_LFn','HRV_LFHF'], axis = 1)
    data[['HRV_HF', 'HRV_VHF', 'HRV_HFn', 'HRV_LnHF', 'HRV_MSE', 'HRV_CMSE', 'HRV_RCMSE']] = data[['HRV_HF', 'HRV_VHF', 'HRV_HFn', 'HRV_LnHF', 'HRV_MSE', 'HRV_CMSE', 'HRV_RCMSE']].replace(np.nan, 0) 
    data = data.dropna(axis='columns')
    features = data
    features.replace([np.inf, -np.inf], np.nan, inplace=True)
    features = features.dropna(axis='columns')
    #features.drop(columns=features.columns[:2], axis=1,  inplace=True)
    #features = features.reset_index()
    return features

In [9]:
def sensorPPGfilter(data):
    fs = 400
    ir_data = data.iloc[100:,[3]]

    plt.figure(figsize=(20,4))
    #plt.plot(ir_data)
    # labels and titles
    #plt.title('Raw IR signal')
    #plt.xlabel('$Index$') 
    #plt.ylabel('$PPG$') 
    #plt.figure(figsize=(20,4))

    ir_data_clean = nk.ppg_clean(ir_data, method='nabian2018', sampling_rate=1000)

    # Parameters 
    order = 4
    frequency = np.array([0.5,10])
    sampling_rate = 300
    frequency = frequency / (sampling_rate / 2)  # Normalize frequency to Nyquist Frequency (Fs/2).

    # Filtering
    b, a = scipy.signal.butter(order, frequency, btype='band')
    ir_filt = scipy.signal.filtfilt(b, a, ir_data_clean, axis=0)

    peaks_red, _ = find_peaks(np.array(ir_filt), height=0, distance=0.5*fs)
    peaks_red_min, _ = find_peaks(-ir_filt, height=0, distance=0.5*fs)

    ir_filt = scipy.signal.filtfilt(b, a, ir_data_clean, axis=0)[peaks_red_min[0]-1:peaks_red_min[-1]-1]
    peaks_red, _ = find_peaks(np.array(ir_filt), height=0, distance=0.5*fs)
    peaks_red_min, _ = find_peaks(-ir_filt, height=0, distance=0.5*fs)

    # plot
    plt.figure(figsize=(20,4))
    #plt.plot((red_data-red_data.iloc[0,0]))
    plt.plot(-ir_filt)
    #plt.plot(peaks_red, red_filt[peaks_red], "x")
    #plt.plot(peaks_red_min, red_filt[peaks_red_min], "x")
    # labels and titles
    plt.title('Filtered IR signal')
    plt.xlabel('$Index$') 
    plt.ylabel('$PPG$') 
    return -ir_filt